In [6]:
%load_ext autoreload
%autoreload 2

from ff_energy.utils import pickle_output, read_from_pickle
import pandas as pd
import scikit_posthocs as sp
import seaborn as sns
import matplotlib.pyplot as plt
from ff_energy.plot import plot_energy_MSE, plot_ff_fit

from ff_energy.structure import atom_key_pairs
from ff_energy.potential import LJ, akp_indx
from ff_energy.ff import FF
from ff_energy.ff_fit import LJ_bound, load_ff
from ff_energy.data import pairs_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#  Data 
##  ab intio data

In [3]:
pc_pbe0_dz_d4 = next(read_from_pickle("data_pc_pbe0_d4.obj.pkl"))
pc_pbe0_dz = next(read_from_pickle("data_pc_pbe0.obj.pkl"))
pc_hf_dz = next(read_from_pickle("data_pc_hfdz.obj.pkl"))
pc_pbe0tz = next(read_from_pickle("data_pc_pbe0tz.obj.pkl"))
pnolccsd_pvtzdf = next(read_from_pickle("data_pc_pno-lccsd-pvtzdf.obj.pkl"))

In [12]:
pc_pbe0_dz.monomer_df

,M_ENERGY,KEY,n_monomers
test467,-1527.154217,test467,20
test263,-1527.156016,test263,20
test35,-1527.143158,test35,20
test223,-1527.147556,test223,20
test254,-1527.135568,test254,20
...,...,...,...
test100,-1527.141372,test100,20
test304,-1527.150985,test304,20
test327,-1527.154084,test327,20
test185,-1527.144999,test185,20


#  FFs

In [ ]:
test_ff = load_ff("test", 
        "water_cluster", 
        pk = "data_pc_pbe0.obj.pkl",
        intern="harmonic",
        elec="ECOL"
       )

Pickled structures/dists found:  ['water_cluster.pkl.pkl', 'water_dimer.pkl.pkl', 'water_dimer.pkl', 'structures.pkl', 'dists.pkl', 'water_cluster.pkl']
Loading pickled distances/structure: water_cluster
loading data from pickle: data_pc_pbe0.obj.pkl
